Considere novamente o problema de atrasos de vôos, disponível em <https://www.kaggle.com/usdot/flight-delays>. Nesta atividade, além dos dados de `flights.csv`, nós iremos utilizar informações disponíveis nos arquivos `airlines.csv` e `airports.csv`.

1.  Importe, utilizando o pacote `readr`, cada um dos três arquivos disponíveis. Os objetos resultantes devem ser chamados `flights`, `airlines` e `airports`.

    -   Para o arquivo de aeroportos, importe apenas as colunas `IATA_CODE`, `CITY`, `STATE`, `LATITUDE`, `LONGITUDE`;
    -   Para o arquivo de vôos:
        a.  Importe apenas as colunas `DESTINATION_AIRPORT` e `ARRIVAL_DELAY`;
        b.  Leia apenas 1 milhão de linhas por vez;
        c.  Remova vôos em que o aeroporto de destino comece com a letra 1;
        d.  Remova registros em que existam pelo menos uma coluna faltante;
        e.  Determine, para cada parte do arquivo, as estatísticas suficientes para a determinação do atraso médio por aeroporto de destino;
        f.  Ao finalizar a leitura do arquivo, combine as estatísticas suficientes de modo a produzir a média de atraso por aeroporto.


In [ ]:
getStats = function(input, pos){
  input %>%
    filter(!startsWith(DESTINATION_AIRPORT, "1")) %>%
    drop_na() %>%
    group_by(DESTINATION_AIRPORT) %>%
    summarise(Sx=sum(ARRIVAL_DELAY), n=n())
}

computeStats = function(input){
  input %>%
    group_by(DESTINATION_AIRPORT) %>%
    summarise(MEAN_DELAY = sum(Sx)/sum(n))
}

mycolsf = cols_only(DESTINATION_AIRPORT='c', ARRIVAL_DELAY='i')
flights <- read_csv_chunked("../dados/flights.csv.zip",
                            callback=DataFrameCallback$new(getStats),
                            chunk_size = 1e6,
                            col_types = mycolsf)

# flights <- read_csv_chunked("flights.csv",
#                             callback=DataFrameCallback$new(getStats),
#                             chunk_size = 1e6,
#                             col_types = mycolsf)

flights %<>% computeStats()

airlines <- read_csv("../dados/airlines.csv")
## airlines <- read_csv("airlines.csv")
mycolsp = cols_only(IATA_CODE='c', CITY='c', STATE='c', LATITUDE='d', LONGITUDE='d')
airports <- read_csv("../dados/airports.csv", col_types = mycolsp)
## airports <- read_csv("airports.csv", col_types = mycolsp)

ERROR: Error in fileUpload(): could not find function "fileUpload"


2.  Selecione a operação apropriada *join* para incluir, na tabela `flights`, as colunas `CITY` e `STATE` do objeto `airports`. Para executar esta tarefa:
    a.  Identifique a coluna que é a chave na tabela `flights`;
    b.  Identifique a coluna que é a chave na tabela `airports`;
    c.  Quais são os aeroportos que estão listados em `flights`, mas estão ausentes em `airports`?
    d.  Apresente o comando que combine ambas as tabelas, indicando explicitamente as chaves;
    e.  Armazene a tabela resultante no objeto `flights`.

In [ ]:
## chave em flights: DESTINATION_AIRPORT
## chave em airports: IATA_CODE
flights %>% anti_join(airports, by=c('DESTINATION_AIRPORT'='IATA_CODE'))
flights %<>%
  left_join(airports, by=c('DESTINATION_AIRPORT'='IATA_CODE'))

3.  Quantos aeroportos cada estado possui? Apresente uma tabela ordenada de forma decrescente (no número de aeroportos).


In [ ]:
flights %>% group_by(STATE) %>% tally() %>% arrange(-n)

4.  Apresente um mapa representando todos os atrasos observados por aeroporto.

    a.  Carregue o pacote `leaflet`;
    b.  Combine os comandos `leaflet`, `addTiles` e `addMarkers` para a criação de um mapa básico;
    c.  Armazene o grafico em b) numa variável chamada `this`;
    d.  Adicione as duas linhas abaixo após a criação da variável `this` (apenas se você estiver usando Jupyter):


In [ ]:
library(leaflet)
this = leaflet(flights) %>% addTiles() %>%
    addCircleMarkers(~LONGITUDE, ~LATITUDE,
                     radius=~MEAN_DELAY,
                     popup=~paste("Atraso médio:", as.character(round(MEAN_DELAY, 0)), "min"),
                     label=~as.character(DESTINATION_AIRPORT),
                     fillOpacity=0.5,
                     color=~ifelse(MEAN_DELAY < 10, "navy", "red"),
                     clusterOptions=markerClusterOptions())